# Bidirectional Vanilla RNN corpus Otomi

In [1]:
from reccurrent_model import biRNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pickle import load
from itertools import chain
import pandas as pd
import numpy as np
import torch

In [2]:
file1 = load(open('pickle_objects/preinput_data','rb'))
print(file1[0], len(file1))

[[["'", 'v', 'B-como'], ['á', 'v', 'I-como'], ['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['m', 'v', 'B-ctrf'], ['á', 'v', 'I-ctrf'], ['k', 'v', 'B-stem'], ['h', 'v', 'I-stem'], ['á', 'v', 'I-stem']]] 1769


In [3]:
data = []

for s in file1:
    sent_data = []
    for w in s:
        #print(w)
        word = [(x[0], x[2]) for x in w] + [(' ', ' ')]
        sent_data.append(word)
    
    one_sent = list(chain(*sent_data))
    one_sent.pop(-1)
    data.append(list(zip(*one_sent)))

In [4]:
#Data split 33%
train_pairs, eval_pairs = train_test_split(data, test_size=0.33)
print(len(train_pairs), len(eval_pairs))

#Separar el train en dos conjuntos
train_sents, train_tags = zip(*train_pairs)

1185 584


In [5]:
model = biRNN(train_sents, train_tags)

In [6]:
%%time
model.train_model(its=150)

100%|██████████| 150/150 [28:17<00:00, 11.32s/it]

CPU times: user 5h 3min 7s, sys: 51.6 s, total: 5h 3min 59s
Wall time: 28min 17s


### Evaluation

In [7]:
y_pred = []
y_true = []
for sent in eval_pairs:
    prediccion = model.forward(sent[0])
    y_pred.append(prediccion)
    y_true.append(sent[1])

In [8]:
y_true = list(chain(*y_true))
y_pred = list(chain(*y_pred))

In [9]:
labels = list(set(y_true))
labels.remove(' ')

print('Accuracy:', accuracy_score(y_true, y_pred))

Accuracy: 0.7415092009386193


In [10]:
prec, rec, f1, supp = precision_recall_fscore_support(y_true, y_pred, labels=labels)

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
results = pd.DataFrame(data=np.array([prec, rec, f1, supp]).T, index=labels, columns=['Precision','Recall','F1', 'Support'])
results

,Precision,Recall,F1,Support
I-que,0.000000,0.000000,0.000000,4.0
I-2.prf,0.000000,0.000000,0.000000,2.0
B-agujerear/v,0.000000,0.000000,0.000000,1.0
B-1.cpl,0.000000,0.000000,0.000000,59.0
B-aqui,0.000000,0.000000,0.000000,3.0
...,...,...,...,...
I-toluca,0.000000,0.000000,0.000000,24.0
B-1.obj,0.000000,0.000000,0.000000,41.0
B-det,0.699681,0.935897,0.800731,234.0
I-encl,0.000000,0.000000,0.000000,1.0


In [12]:
print(results.to_string())

                  Precision    Recall        F1  Support
I-que              0.000000  0.000000  0.000000      4.0
I-2.prf            0.000000  0.000000  0.000000      2.0
B-agujerear/v      0.000000  0.000000  0.000000      1.0
B-1.cpl            0.000000  0.000000  0.000000     59.0
B-aqui             0.000000  0.000000  0.000000      3.0
B-que              0.000000  0.000000  0.000000      4.0
B-1.cnt            0.343284  1.000000  0.511111     23.0
I-1.cnt            0.326241  1.000000  0.491979     46.0
B-3.pls            0.000000  0.000000  0.000000     20.0
I-dem              0.000000  0.000000  0.000000     90.0
I-3.obj            0.000000  0.000000  0.000000     21.0
I-adj              0.000000  0.000000  0.000000      1.0
I-1.pls            0.000000  0.000000  0.000000      4.0
B-stem             0.683049  0.971020  0.801967   2519.0
B-p.loc            0.000000  0.000000  0.000000      6.0
I-1.pot            0.000000  0.000000  0.000000     67.0
I-3.cpl            0.517986  1.

In [13]:
aver_prec = (supp*prec).sum()/supp.sum()
aver_rec = (supp*rec).sum()/supp.sum()
aver_f1 = (supp*f1).sum()/supp.sum()

print('Average precision: {}\nAverage recall: {}\nAverage F1: {}'.format(aver_prec, aver_rec, aver_f1))

Average precision: 0.5046193346273097
Average recall: 0.6992600043106545
Average F1: 0.583584752114077


In [14]:
#torch.save(model,'Model.biLSTM.justChar')

#### Save embeddings

In [15]:
embs = {}
for w,v in model.input_voc.items():
    if w != ' ':
        vec = model.emb[0](torch.tensor([v])).detach().numpy()[0]
        embs[w] = vec   

In [16]:
from pickle import dump

dump( embs, open( "Embs.biRNN.justChar.p", "wb" ) )